In [1]:
import os
import json
import requests

from web3 import Web3, HTTPProvider

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

from utils.eth_blocks_utils import get_block_for_timestamp, get_timestamp_for_block

from pandas import Timestamp

In [2]:
ALCHEMY_API_KEY = os.environ['ALCHEMY_API_KEY']
web3 = Web3(HTTPProvider(f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}"))

In [3]:
api = "https://api.thegraph.com/subgraphs/name/convex-community/volume-mainnet"

In [4]:
mim3crv = "0x5a6A4D54456819380173272A5E8E9B9904BdF41B"
with open("./mim-3pool.json", 'r') as f:
    abi = json.load(f)
mim_3pool = web3.eth.contract(address=mim3crv, abi=abi)

wormholeust3crv = "0xCEAF7747579696A2F0bb206a14210e3c9e6fB269"
with open("./wormholeust-3pool.json", 'r') as f:
    abi = json.load(f)
wormholeust_3pool = web3.eth.contract(address=wormholeust3crv, abi=abi)

usdn3crv = "0x0f9cb53Ebe405d49A0bbdBD291A65Ff571bC83e1"
with open("./usdn-3pool.json", 'r') as f:
    abi = json.load(f)
usdn_3pool = web3.eth.contract(address=usdn3crv, abi=abi)

pools = [
    "0x5a6A4D54456819380173272A5E8E9B9904BdF41B",
    "0xCEAF7747579696A2F0bb206a14210e3c9e6fB269",
    "0x0f9cb53Ebe405d49A0bbdBD291A65Ff571bC83e1"
]

In [29]:
end_block = web3.eth.get_block_number()
end_block_timestamp = get_timestamp_for_block(end_block-1)
end_block_timestamp

1652342427

MIM was the first depeg so we start our queries from around mid Feb

In [6]:
depegs_start = int(Timestamp(2022, 2, 15).timestamp())
query_start_block = get_block_for_timestamp(depegs_start)
query_start_block

14207339

In [30]:
step = 10000
time_steps = list(range(depegs_start, end_block_timestamp, step))
time_boxes = list(zip(time_steps, time_steps[1:]))
len(time_boxes)

745

In [ ]:
blocks_to_inspect = []
c = len(time_boxes)
for time_box in time_boxes:
    print(f"left: {c}")
    c -= 1
    for pool in pools:
        query = f"""
        {{
          swapEvents(
            where: {{
              pool: "{pool.lower()}"
              timestamp_gte: {time_box[0]}
              timestamp_lt: {time_box[1]}
            }}
          ) {{
            block
          }}
        }}
        """
        r = requests.post(api, json={'query': query})
        queried_data = [int(i['block']) for i in dict(r.json())['data']['swapEvents']]
        blocks_to_inspect.extend(queried_data)
        data = list(set(blocks_to_inspect))

blocks_to_inspect

Get blocks wherever transactions happened.

In [33]:
num_blocks = len(blocks_to_inspect)
num_blocks

14670

We want token reserves and spot prices at swap_blocks (but first remove duplicates!)

In [21]:
data = []
done_blocks = []

In [ ]:
dec_18 = 1000000000000000000
dec_6 = 1000000
for block in blocks_to_inspect:

    if block in done_blocks:
        continue

    print(f"blocks left: {num_blocks - len(done_blocks)}")

    block_timestamp = pd.Timestamp.fromtimestamp(get_timestamp_for_block(block))

    # spot prices - outputs are in 3crv so 18 decimals are divided
    mim_spot_price = mim_3pool.functions.get_dy(0, 1, dec_18).call(block_identifier=block) * 1e-18
    wormholeust_spot_price = wormholeust_3pool.functions.get_dy(0, 1, dec_6).call(block_identifier=block) * 1e-18
    usdn_spot_price = usdn_3pool.functions.get_dy(0, 1, dec_18).call(block_identifier=block) * 1e-18

    # reserves
    mim_reserve = mim_3pool.functions.balances(0).call(block_identifier=block) * 1e-18
    mim_3crv_reserve = mim_3pool.functions.balances(1).call(block_identifier=block) * 1e-18

    wormholeust_reserve = wormholeust_3pool.functions.balances(0).call(block_identifier=block) * 1e-6
    wormholeust_3crv_reserve = wormholeust_3pool.functions.balances(1).call(block_identifier=block) * 1e-18

    usdn_reserve = usdn_3pool.functions.balances(0).call(block_identifier=block) * 1e-18
    usdn_3crv_reserve = usdn_3pool.functions.balances(1).call(block_identifier=block) * 1e-18

    data.append(
        {
            "block": block,
            "timestamp": block_timestamp,
            "mim_spot_price": mim_spot_price,
            "wormholeust_spot_price": wormholeust_spot_price,
            "usdn_spot_price": usdn_spot_price,
            "mim_reserve": mim_reserve,
            "mim_3crv_reserve": mim_3crv_reserve,
            "wormholeust_reserve": wormholeust_reserve,
            "wormholeust_3crv_reserve": wormholeust_3crv_reserve,
            "usdn_reserve": usdn_reserve,
            "usdn_3crv_reserve": usdn_3crv_reserve,
        }
    )
    done_blocks.append(block)

In [ ]:
df_data = pd.DataFrame(data)
df_data

In [31]:
df_data.to_csv("./mim_usdn_wormhole_depeg_reserves_spot_price.csv")